In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np

In [351]:
##L'heure et la date de la requête : 
import time
import datetime
#On regarde l'heure et la date de la requête
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()) #Date et heure en entier
Date = time.strftime('%Y-%m-%d', time.localtime())
Time = time.strftime('%H:%M', time.localtime()) #Heure uniquement
DateTime = time.strftime('%Y-%m-%dT%H:%M:%S', time.localtime()) 
# Pour faciliter ici, je n'ai pas arrondi les minutes, mais rogné
#Je suis partie du postulat que pour nos données, la précision n'était pas ce qui était  le principal à rechercher
print('Voic la date de la recherche : ', Date)
print("Voici l'heure de la recherche :", Time)
print(DateTime, 'heure')

Voic la date de la recherche :  2020-12-09
Voici l'heure de la recherche : 21:34
2020-12-09T21:34:40 heure


On prélève les données géopandas sur le site : https://www.data.gouv.fr/fr/datasets/prevision-meteo-ile-de-france-arome/#_

## la météo en temps réel

In [217]:
meteo_gpd = 'https://www.data.gouv.fr/fr/datasets/r/8e834f69-0b43-439a-87bd-87ca80c84cd7'
meteo_idf = gpd.read_file(meteo_gpd) 
#Seule base de donnée avec prévisions pour les 36h après sa mise à jour (seule gratuite)
#En plus : On peut planifier la météo pour le trajet à quelques heures près dans la journée qui arrive
#Donne la prévision qui a été faite la veille à 13h.

In [352]:
meteo_idf.head(5)

,timestamp,forecast,2_metre_temperature,total_water_precipitation,geometry,duration
18333,2020-12-09T13:00:00,2020-12-09T13:00:00,2.658228,NaN,POINT (2.35000 48.85000),0.0
18394,2020-12-09T13:00:00,2020-12-09T14:00:00,2.351099,0.000244,POINT (2.35000 48.85000),60.0
18932,2020-12-09T13:00:00,2020-12-09T15:00:00,2.784174,0.001465,POINT (2.35000 48.85000),120.0
19445,2020-12-09T13:00:00,2020-12-09T16:00:00,2.949854,0.193848,POINT (2.35000 48.85000),180.0
18336,2020-12-09T13:00:00,2020-12-09T17:00:00,3.238123,0.528809,POINT (2.35000 48.85000),240.0


D'après la documentation, 

- La colonne "total_water_precipitation" correspond au cumul (depuis le début de la simulation) de la somme des flux au sol des précipitations liquides et solides (eau, neige, grêlons) en kg/m-2
- La température est celle diagnostiquée à 2m de hauteur au dessus du relief . Ce paramètre peut a
priori être comparé aux mesures faites in situ (stations d’observations humaines ou automatiques).

In [353]:
meteo_idf["duration"] = pd.to_datetime(meteo_idf["forecast"]) - pd.to_datetime(meteo_idf["timestamp"])
meteo_idf["duration"] = meteo_idf["duration"].astype("timedelta64[m]")
#Différence en minutes entre la prévision et l'heure prévue
meteo_idf = meteo_idf.cx[2.35,48.85] 
#On ne garde que les prévisions pour le centre de Paris ce qu'on suppose signifiant pour le reste de la région
meteo_idf = meteo_idf.sort_values('forecast')

In [354]:
meteo_idf2 = meteo_idf.reset_index()

la colonne "total_water_precipitation" correspond à la somme des précipitations depuis la date de référence

In [355]:
for i in range(1,len(meteo_idf2)):
    meteo_idf2.loc[i, 'diff_precipitation'] = meteo_idf2.loc[i,'total_water_precipitation']-meteo_idf2.loc[i-1,'total_water_precipitation']
#On remarque qu'il y a parfois des erreurs dans la conception des données, lorsque "durée" est négative 
#En effet, cela voudrait dire que la somme des précipitations a diminué entre deux heures, ce qui est impossible
#Pour négliger ces erreurs, on remplace les valeurs négatives par 0
meteo_idf2.loc[(meteo_idf2["diff_precipitation"]<0), "diff_precipitation"] = 0

In [356]:
meteo_idf2.sample(3)

,index,timestamp,forecast,2_metre_temperature,total_water_precipitation,geometry,duration,diff_precipitation
25,17973,2020-12-09T13:00:00,2020-12-10T14:00:00,5.220605,0.539063,POINT (2.35000 48.85000),1500.0,0.000000
5,18339,2020-12-09T13:00:00,2020-12-09T18:00:00,3.605676,0.529785,POINT (2.35000 48.85000),300.0,0.000976
33,18403,2020-12-09T13:00:00,2020-12-10T22:00:00,5.046655,0.576172,POINT (2.35000 48.85000),1980.0,0.000000


On pourrait faire un prolongement, en regardant exactement la localisation du point de départ et d'arrivée, mais bon, déjà, j'essyae avec ça

In [357]:
for i in range(1,len(meteo_idf2)):
    meteo_idf2.loc[i, 'duree_intervalle'] = meteo_idf2.loc[i,'duration']-meteo_idf2.loc[i-1,'duration']
#Les intervalles sont souvent d'1h, mais parfois réalisés avec 2h

Je convertis les données de précipitations qui sont en kg/m-2 en données de précipitations en mm/h :

1kg d'eau = 1L d'eau, or 1L/m² =1mm, donc avec le passage des minutes aux heures, on obtient : 
En une heure, on compte 
- (précipitations pendant l'intervalle)/(duree intervalle)*60 

La manière de mesurer les précipitations la plus répandue est la pluviométrie, qui correspond à la hauteur d'eau recueillie sur une surface plane (exprimée en mm). 
On sépare l'intensité de la pluie : 
- pluie très faible et négligeable  (de 0 à 2 mm/h), 
- modérée et forte (plus 2 mm/h)

In [359]:
meteo_idf2["precipitations mm/h"] = ((meteo_idf2["diff_precipitation"])/(meteo_idf2["duree_intervalle"]))*60
##A vérifier
meteo_idf3 = meteo_idf2[['forecast', '2_metre_temperature','precipitations mm/h']]
meteo_idf3.sort_values('forecast').sample(4)

,forecast,2_metre_temperature,precipitations mm/h
10,2020-12-09T23:00:00,2.901086,0.000977
0,2020-12-09T13:00:00,2.658228,NaN
39,2020-12-11T04:00:00,4.427881,0.000000
32,2020-12-10T21:00:00,4.776147,0.000000


In [360]:
meteo_idf_temps_reel = meteo_idf2.loc[meteo_idf2['forecast'] <= DateTime][['forecast', '2_metre_temperature', 'precipitations mm/h' ]].sort_values('forecast').tail(1)
# On fait en rognant l'heure et non en approximant => voir si améliorable ou pas
meteo_idf_temps_reel

,forecast,2_metre_temperature,precipitations mm/h
8,2020-12-09T21:00:00,3.082666,0.0


In [361]:
if meteo_idf_temps_reel['precipitations mm/h'].item()> 2.000 and (meteo_idf_temps_reel['2_metre_temperature'].item() > -1 or meteo_idf_temps_reel['2_metre_temperature'].item()) < 1:
    print('Attention il pleut et il y a peut-être du verglas sur la chaussée.')
elif meteo_idf_temps_reel['precipitations mm/h'].item() > 2.000 and (meteo_idf_temps_reel['2_metre_temperature'].item() < -1 or meteo_idf_temps_reel['2_metre_temperature'].item()) > 1:
    print("Attention il pleut, mais normalement, il n'y a pas de verglas sur la chaussée")
elif meteo_idf_temps_reel['precipitations mm/h'].item() < 2.00 and (meteo_idf_temps_reel['2_metre_temperature'].item() > -1 or meteo_idf_temps_reel['2_metre_temperature'].item()) < 1:
    print("Il ne pleut pas, mais attention, il peut y avoir du verglas.")
else:
    print("Quel chanceux, il n'y a ni pluie ni verglas.")

Quel chanceux, il n'y a ni pluie ni verglas.


C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `item` has been deprecated and will be removed in a future version
  """Entry point for launching an IPython kernel.
C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: `item` has been deprecated and will be removed in a future version
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: `item` has been deprecated and will be removed in a future version
  """


### Les heures de lever et de coucher du soleil : 

Difficulté pour trouver une base de donnée propre : seulement à télécharger par année
On prend la base de donnée sur le site : http://ptaff.ca/, et je l'ai mise sur Github

In [362]:
heures_soleil = pd.read_csv('D:/Données/ENSAE/lever_coucher_soleil.csv',  sep = ',')
heures_soleil.loc[heures_soleil['Date'] == Date] 

,Date,Lever_du_soleil,Coucher_du_soleil,Durée_du_jour
343,2020-12-09,08:32,16:54,08:22


In [363]:
heures_soleil.loc[heures_soleil['Date'] == Date]
if  Time >=heures_soleil.loc[heures_soleil['Date'] == Date]['Lever_du_soleil'].any() and Time <=heures_soleil.loc[heures_soleil['Date'] == Date]['Coucher_du_soleil'].any():
    print('Vous roulez de jour')
else: 
    print('Attention, vous roulez de nuit')

Attention, vous roulez de nuit


## Modifications de la base accident vélo
Le but est de regrouper les informations

In [124]:
accident_velo = pd.read_csv('https://www.data.gouv.fr/fr/datasets/r/3d5f2317-5afd-4a9f-a9c5-bd4fe0113f39')
accident_velo_idf = accident_velo[accident_velo['departement'].isin(['75','77','78','91','92','93','94','95'])]
# Il y a un problème avec les latitudes et les longitudes nulles et les quelques erreurs

C:\Users\Vinciane\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (5,6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [160]:
accident_velo_idf_meteo = accident_velo_idf[['identifiant accident','date','mois', 'jour', 'heure', 'departement','luminosite', 'conditions atmosperiques', 'etat surface', 'sexe', 'age', 'gravite accident', 'commune', ]]
accident_velo_idf_meteo

,identifiant accident,date,mois,jour,heure,departement,luminosite,conditions atmosperiques,etat surface,sexe,age,gravite accident,commune
58,200500001606,2005-01-13,01 - janvier,3 - jeudi,11.0,78,Plein jour,Normale,normale,M,61-62,2 - Blessé hospitalisé,78230
61,200500001930,2005-01-15,01 - janvier,5 - samedi,15.0,95,Plein jour,Normale,normale,M,58-59,0 - Indemne,95042
221,200500004591,2005-01-04,01 - janvier,1 - mardi,17.0,75,Nuit avec éclairage public allumé,Normale,normale,M,36-37,1 - Blessé léger,75001
222,200500004597,2005-01-03,01 - janvier,0 - lundi,9.0,75,Plein jour,Normale,normale,M,26-27,0 - Indemne,75005
223,200500004600,2005-01-03,01 - janvier,0 - lundi,17.0,75,Nuit avec éclairage public allumé,Normale,normale,M,31-32,0 - Indemne,75019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65531,201800050291,2018-07-10,07 - juillet,1 - mardi,9.0,75,Plein jour,Normale,normale,F,58-59,2 - Blessé hospitalisé,75101
65532,201800050337,2018-11-17,11 - novembre,5 - samedi,12.0,75,Plein jour,Normale,normale,M,59-60,1 - Blessé léger,75101
65533,201800050355,2018-02-01,02 - février,3 - jeudi,15.0,75,Plein jour,Temps couvert,mouillée,M,31-32,1 - Blessé léger,75102
65534,201800050359,2018-02-12,02 - février,0 - lundi,9.0,75,Plein jour,Normale,normale,F,29-30,1 - Blessé léger,75101


Le but, c'est d'utiliser les 18 000 lignes, donc on ne va pas utiliser latitude et longitude (absence d'information)
Si j'ai le temps sinon, j'indiquerai un point grossier avec le code commune d'une latitude et longitude pour avoir une météo précise, mais ca c'est si j'ai le temps

In [126]:
#En ile de france
accident_velo_idf.groupby('jour')["identifiant accident"].count()
#Pour ca il faut utiliser les compteurs vélo

jour
0 - lundi       2497
1 - mardi       2798
2 - mercredi    2921
3 - jeudi       2842
4 - vendredi    2683
5 - samedi      1980
6 - dimanche    1820
Name: identifiant accident, dtype: int64

In [127]:
accident_velo_idf_meteo['luminosite'][accident_velo_idf_meteo['luminosite'] == 'Plein jour'] = 'Jour'
accident_velo_idf_meteo['luminosite'][accident_velo_idf_meteo['luminosite'] == 'Nuit avec éclairage public allumé'] = 'Nuit'
accident_velo_idf_meteo['luminosite'][accident_velo_idf_meteo['luminosite'] == 'Nuit avec éclairage public non allumé'] = 'Nuit'
accident_velo_idf_meteo['luminosite'][accident_velo_idf_meteo['luminosite'] == 'Nuit sans éclairage public'] = 'Nuit'
accident_velo_idf_meteo['luminosite'][accident_velo_idf_meteo['luminosite'] == 'Crépuscule ou aube'] = 'Nuit'

C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Vinciane\Anaconda3\lib\site-packages\pandas\core\generic.py:9114: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)
C:\Users\Vinciane\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3326: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-c

In [128]:
accident_velo_idf_meteo.groupby('luminosite')["identifiant accident"].count()

luminosite
Jour    13727
Nuit     3814
Name: identifiant accident, dtype: int64

In [350]:
acc_copy = accident_velo_idf_meteo.copy()
acc_copy[acc_copy['conditions atmosperiques'] == 'Normale']['conditions atmosperiques'] = 'Normale'
acc_copy

C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,identifiant accident,date,mois,jour,heure,departement,luminosite,conditions atmosperiques,etat surface,sexe,age,gravite accident,commune
58,200500001606,2005-01-13,01 - janvier,3 - jeudi,11.0,78,Plein jour,Normale,normale,M,61-62,2 - Blessé hospitalisé,78230
61,200500001930,2005-01-15,01 - janvier,5 - samedi,15.0,95,Plein jour,Normale,normale,M,58-59,0 - Indemne,95042
221,200500004591,2005-01-04,01 - janvier,1 - mardi,17.0,75,Nuit avec éclairage public allumé,Normale,normale,M,36-37,1 - Blessé léger,75001
222,200500004597,2005-01-03,01 - janvier,0 - lundi,9.0,75,Plein jour,Normale,normale,M,26-27,0 - Indemne,75005
223,200500004600,2005-01-03,01 - janvier,0 - lundi,17.0,75,Nuit avec éclairage public allumé,Normale,normale,M,31-32,0 - Indemne,75019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65531,201800050291,2018-07-10,07 - juillet,1 - mardi,9.0,75,Plein jour,Normale,normale,F,58-59,2 - Blessé hospitalisé,75101
65532,201800050337,2018-11-17,11 - novembre,5 - samedi,12.0,75,Plein jour,Normale,normale,M,59-60,1 - Blessé léger,75101
65533,201800050355,2018-02-01,02 - février,3 - jeudi,15.0,75,Plein jour,Couvert,mouillée,M,31-32,1 - Blessé léger,75102
65534,201800050359,2018-02-12,02 - février,0 - lundi,9.0,75,Plein jour,Normale,normale,F,29-30,1 - Blessé léger,75101


Pour résoudre l'erreur : cf ce site : https://www.dataquest.io/blog/settingwithcopywarning/

In [148]:
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Normale'] = 'Normale'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Temps éblouissant'] = 'Normale'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Brouillard - fumée'] = 'Couvert'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Temps couvert'] = 'Couvert'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Pluie forte'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Pluie légère'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Neige - grêle'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Vent fort - tempête'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Autre'] = 'Normale'
accident_velo_idf_meteo

C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel packag

,identifiant accident,date,mois,jour,heure,departement,luminosite,conditions atmosperiques,etat surface,sexe,age,gravite accident,commune
58,200500001606,2005-01-13,01 - janvier,3 - jeudi,11.0,78,Jour,Normale,normale,M,61-62,2 - Blessé hospitalisé,78230
61,200500001930,2005-01-15,01 - janvier,5 - samedi,15.0,95,Jour,Normale,normale,M,58-59,0 - Indemne,95042
221,200500004591,2005-01-04,01 - janvier,1 - mardi,17.0,75,Nuit,Normale,normale,M,36-37,1 - Blessé léger,75001
222,200500004597,2005-01-03,01 - janvier,0 - lundi,9.0,75,Jour,Normale,normale,M,26-27,0 - Indemne,75005
223,200500004600,2005-01-03,01 - janvier,0 - lundi,17.0,75,Nuit,Normale,normale,M,31-32,0 - Indemne,75019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65531,201800050291,2018-07-10,07 - juillet,1 - mardi,9.0,75,Jour,Normale,normale,F,58-59,2 - Blessé hospitalisé,75101
65532,201800050337,2018-11-17,11 - novembre,5 - samedi,12.0,75,Jour,Normale,normale,M,59-60,1 - Blessé léger,75101
65533,201800050355,2018-02-01,02 - février,3 - jeudi,15.0,75,Jour,Couvert,mouillée,M,31-32,1 - Blessé léger,75102
65534,201800050359,2018-02-12,02 - février,0 - lundi,9.0,75,Jour,Normale,normale,F,29-30,1 - Blessé léger,75101


In [161]:
accident_velo_idf_meteo['conditions atmosperiques'].loc[accident_velo_idf_meteo['conditions atmosperiques'] == 'Normale']= 'Normale'
accident_velo_idf_meteo['conditions atmosperiques'].loc[accident_velo_idf_meteo['conditions atmosperiques'] == 'Temps éblouissant'] = 'Normale'
accident_velo_idf_meteo['conditions atmosperiques'].loc[accident_velo_idf_meteo['conditions atmosperiques'] == 'Temps couvert'] = 'Couvert'
accident_velo_idf_meteo['conditions atmosperiques'].loc[accident_velo_idf_meteo['conditions atmosperiques'] == 'Pluie forte'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'].loc[accident_velo_idf_meteo['conditions atmosperiques'] == 'Pluie légère'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'].loc[accident_velo_idf_meteo['conditions atmosperiques'] == 'Neige - grêle'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'].loc[accident_velo_idf_meteo['conditions atmosperiques'] == 'Vent fort - tempête'] = 'Precipitations'
accident_velo_idf_meteo['conditions atmosperiques'].loc[accident_velo_idf_meteo['conditions atmosperiques'] == 'Autre'] = 'Normale'
accident_velo_idf_meteo

C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Vinciane\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel packag

,identifiant accident,date,mois,jour,heure,departement,luminosite,conditions atmosperiques,etat surface,sexe,age,gravite accident,commune
58,200500001606,2005-01-13,01 - janvier,3 - jeudi,11.0,78,Plein jour,Normale,normale,M,61-62,2 - Blessé hospitalisé,78230
61,200500001930,2005-01-15,01 - janvier,5 - samedi,15.0,95,Plein jour,Normale,normale,M,58-59,0 - Indemne,95042
221,200500004591,2005-01-04,01 - janvier,1 - mardi,17.0,75,Nuit avec éclairage public allumé,Normale,normale,M,36-37,1 - Blessé léger,75001
222,200500004597,2005-01-03,01 - janvier,0 - lundi,9.0,75,Plein jour,Normale,normale,M,26-27,0 - Indemne,75005
223,200500004600,2005-01-03,01 - janvier,0 - lundi,17.0,75,Nuit avec éclairage public allumé,Normale,normale,M,31-32,0 - Indemne,75019
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65531,201800050291,2018-07-10,07 - juillet,1 - mardi,9.0,75,Plein jour,Normale,normale,F,58-59,2 - Blessé hospitalisé,75101
65532,201800050337,2018-11-17,11 - novembre,5 - samedi,12.0,75,Plein jour,Normale,normale,M,59-60,1 - Blessé léger,75101
65533,201800050355,2018-02-01,02 - février,3 - jeudi,15.0,75,Plein jour,Couvert,mouillée,M,31-32,1 - Blessé léger,75102
65534,201800050359,2018-02-12,02 - février,0 - lundi,9.0,75,Plein jour,Normale,normale,F,29-30,1 - Blessé léger,75101


In [149]:
accident_velo_idf_meteo.loc['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Normale'] = 'Normale'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Temps éblouissant'] = 'Normale'
accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Brouillard - fumée'] = 'Couvert'
#accident_velo_idf_meteo_copy = accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Temps couvert'] = 'Couvert'
#accident_velo_idf_meteo_copy = accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Pluie forte'] = 'Precipitations'
#accident_velo_idf_meteo_copy = accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Pluie légère'] = 'Precipitations'
#accident_velo_idf_meteo_copy = accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Neige - grêle'] = 'Precipitations'
#accident_velo_idf_meteo_copy = accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Vent fort - tempête'] = 'Precipitations'
#accident_velo_idf_meteo_copy = accident_velo_idf_meteo['conditions atmosperiques'][accident_velo_idf_meteo['conditions atmosperiques'] == 'Autre'] = 'Normale'
#accident_velo_idf_meteo_copy 

KeyError: 'conditions atmosperiques'

In [130]:
accident_velo_idf_meteo.groupby('conditions atmosperiques')["identifiant accident"].count()

conditions atmosperiques
Couvert             646
Normale           15137
Precipitations     1758
Name: identifiant accident, dtype: int64

## A partir de la, inutile : 